In [ ]:
from vpython import*
from random import random
scene = canvas(width=600, height=500, center=vector(0,0,0),background=vector(0.2,0.5,0))
f1 = gdots(color=color.blue)

##const
c=3e8
k=9e-9
scale=1e-5
size_scale=1e15

##initialization
air=box(pos=vec(0,0,0),length=500,width=500,height=400,opacity=0.5)
total_atoms=100
atoms=[]
photons=[]
for i in range(100):
    atom=sphere(pos=vec(-250+random()*500,-200+random()*400,-250+random()*500),radius=20,color=color.red) ##TODO:扣掉半徑
    atom.v=vec(0,0,0)
    atom.q=0
    atom.mass=32*(1.66e-27)*size_scale
    atoms.append(atom)
    
muons=[]
electrons=[]
for i in range(200):
    muon=sphere(pos=vec(-250+random()*500,199,-250+random()*500),radius=1,color=color.white) ##TODO:扣掉半徑
    muon.v=vec(0,0,0)
    muon.visible=False
    muons.append(muon)
    
photons=[]
for i in range(5000):
    photon=sphere(pos=vec(0,0,0),radius=5,color=color.yellow)
    photon.visible = False
    photons.append(photon)
    
    electron=sphere(pos=vec(-245+random()*(500-10),-195+random()*(400-10),-245+random()*(500-10)),radius=5,color=color.cyan)
    electron.q=-1
    electron.v=vec(0,0,0)
    electron.mass=(9.1e-31)*size_scale
    electrons.append(electron)

def Fe (a,b):
    return ((k*a.q*b.q/mag2(a.pos-b.pos))*norm(a.pos-b.pos)),((k*a.q*b.q/mag2(a.pos-b.pos))*norm(b.pos-a.pos))

t=0
dt=0.01
ele_count=0
light=0
while (t<5):
    rate(10)
    for i in range(100):
        atoms[i].pos+=atoms[i].v*dt
    for i in range(200):
        muons[i].pos+=muons[i].v*dt
        electrons[i].pos+=electrons[i].v*dt
        
    for i in range(200):
        if(muons[i].pos.y==199):
            if(random()<0.2):
                muons[i].visible=True
                muons[i].v=vec(0,0.9*c*scale,0)
        if(muons[i].pos.y<=-199):
            muons[i].visible=False
            muons[i].pos.y=199
            muons[i].v=0
        for j in range(100):
            if(mag(muons[i].pos-atoms[j].pos)>21 and atoms[j].q<2):
                atoms[j].q+=1
                electrons[ele_count].pos=atoms[j].pos+norm(electrons[i].pos-atoms[j].pos)*25
                ##electron speed
                ele_count+=1
                muons[i].visible=False
                muons[i].pos.y=199
                muons[i].v=0
     
    for i in range(200):
        for j in range(100):
            if(mag(atoms[j].pos-electrons[i].pos)<25 and atoms[j].visible==True and electrons[i].visible==True):
                electrons[i].visible=False
                electrons[i].q=0
                atoms[j].q-=1
                photons[light].pos=atoms[j].pos+norm(vec(random(),random(),random()))
                photons[light].visible=True
                f1.plot(photons[light].pos.x,photons[light].pos.z)
                light+=1
      
    for i in range(5000): #consider ion and electron
        for j in range(100):
            Fi,Fj=Fe (electrons[i],atoms[j])
            electrons[i].v+=Fi*dt/electrons[i].mass
            atoms[j].v+=Fj*dt/atoms[j].mass
    for i in range(100): #consider ion and ion
        for j in range(i+1,100):
            Fi,Fj=Fe (atoms[i],atoms[j])
            atoms[i].v+=Fi*dt/atoms[i].mass
            atoms[j].v+=Fj*dt/atoms[j].mass
    for i in range(5000): #consider electron and electron
        for j in range(i+1,5000):
            if(electrons[i].visible==True and electrons[j].visible==True):
                Fi,Fj=Fe (electrons[i],electrons[j])
                electrons[i].v+=Fi*dt/electrons[i].mass
                electrons[j].v+=Fj*dt/electrons[j].mass          
    t+=dt